# Taller pipelines usando el dataset Iris Sucio

#### Grupo:
* Anderson Bornachera
* Juan Mosquera

## Carga del dataset

In [1]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?export=download&id=1Po5q73bcltZ48HQm_ZUwi2LLehxShGzW")
df.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,1,5.1,3.5,1.4,0.2,Setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,NaN,3.6,1.4,0.2,setosa


## Convertimos los nombres de las columnas a snake case.

In [2]:
import re
def to_snake(s):
    s = re.sub(r"[^0-9a-zA-Z]+", "_", s.strip())
    return s.lower()

## Estandarizamos las especies a texto en minúsculas.

In [3]:
import re
def species_to_lower(df):
  df['species'] = df['species'].str.lower()
  return df

## Definimos un transformador personalizado para scikit-learn llamado `CustomDataCleaner`.  
Su función es:  
1. Copiar el DataFrame original para no modificarlo `X.copy()`.  
2. Convertir los nombres de las columnas a `snake_case`.  
3. Pasar a minúsculas los valores de la columna `species`.  

De esta forma, la limpieza de datos puede integrarse fácilmente en un pipeline.
Usamos el pipeline con la variable `custom_cleaner`.

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin

class CustomDataCleaner(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # 1
        df = X.copy()
        # 2
        df.columns = [to_snake(col) for col in df.columns]
        # 3
        df = species_to_lower(df)


        return df

custom_cleaner = CustomDataCleaner()

## Separamos los datos numéricos y categorizamos las especies.

In [5]:
numeric_features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
categorical_features = ['species']

## Se define un pipeline para variables numéricas (`numeric_pipeline`) que:  
1. Imputa valores faltantes con la media (`SimpleImputer(strategy='mean')`).  
2. Escala los datos con `StandardScaler()`.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
from sklearn.impute import SimpleImputer

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

## Se define un pipeline para variables categóricas (`categorical_pipeline`) que:  
1. Imputa valores faltantes con la categoría más frecuente (`SimpleImputer(strategy='most_frequent')`).  
2. Codifica las variables con One-Hot Encoding (`OneHotEncoder`), ignorando categorías desconocidas y eliminando la primera columna para evitar multicolinealidad (`drop='first'`).  

In [7]:
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

## Se crea un `ColumnTransformer` llamado `feature_preprocessor` que aplica:  
- El `numeric_pipeline` a las variables numéricas.  
- El `categorical_pipeline` a las variables categóricas.  
Las columnas no especificadas se descartan (`remainder='drop'`).  

In [8]:
from sklearn.compose import ColumnTransformer

feature_preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
], remainder='drop')

## Se construye un pipeline completo (`complete_pipeline`) que incluye:  
1. `cleaning`: aplica el transformador personalizado (`custom_cleaner`).  
2. `preprocessing`: aplica el preprocesador de características (`feature_preprocessor`).  
3. `classifier`: entrena un modelo `RandomForestClassifier`.  

Finalmente, se imprime la estructura completa del pipeline.

In [9]:
from sklearn.ensemble import RandomForestClassifier

complete_pipeline = Pipeline([
    ('cleaning', custom_cleaner),
    ('preprocessing', feature_preprocessor),
    ('classifier', RandomForestClassifier())
])

print(complete_pipeline)

Pipeline(steps=[('cleaning', CustomDataCleaner()),
                ('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['sepal_length',
                                                   'sepal_width',
                                                   'petal_length',
                                                   'petal_width']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                  

## Se crea una copia del DataFrame (`df_temp`).  
- `X_raw`: contiene todas las columnas (por ahora).
- `y_raw`: contiene la columna objetivo `Species`.  

In [10]:
df_temp = df.copy()
X_raw = df_temp.drop(columns=[])
y_raw = df_temp['Species']

## Se imprimen las dimensiones y los nombres de columnas de `X_raw` para verificar la estructura inicial de los datos.

In [11]:
print(f"\nDatos raw X shape: {X_raw.shape}")
print(f"Columnas X: {X_raw.columns.tolist()}")


Datos raw X shape: (150, 6)
Columnas X: ['Unnamed: 0', 'Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width', 'Species']


## Conversión de clases categóricas en valores numéricos.
Se aplica `LabelEncoder` sobre la variable objetivo (`y_raw`) para convertir las clases categóricas en valores numéricos.  
Además, se muestran las clases detectadas y una muestra de la codificación resultante.  

In [12]:
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y_raw)
print(f"Clases objetivo: {le_target.classes_}")
print(f"y_encoded: {y_encoded[:10]}")

Clases objetivo: ['SETOSA' 'Setosa' 'VERSICOLOR' 'VIRGINICA' 'Versicolor' 'setosa'
 'versicolor' 'virginica']
y_encoded: [1 5 5 5 5 5 5 0 5 5]


## Se divide el dataset en entrenamiento y prueba con `train_test_split`:  
- 80% de los datos para entrenamiento.  
- 20% de los datos para prueba.  
También se imprimen las dimensiones de cada conjunto.

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y_encoded, test_size=0.2, random_state=42
)

print(f"\nTamaño entrenamiento: {X_train.shape}")
print(f"Tamaño prueba: {X_test.shape}")


Tamaño entrenamiento: (120, 6)
Tamaño prueba: (30, 6)


## Entrenamiento del modelo.
Se entrena el pipeline completo con los datos de entrenamiento (`X_train`, `y_train`).  
Esto ajusta el modelo para que aprenda patrones de los datos.



In [14]:
complete_pipeline.fit(X_train, y_train)
print("¡Entrenamiento completado!")

¡Entrenamiento completado!


## Evauluación del modelo.
Se mide la exactitud del modelo en los datos de entrenamiento y de prueba.  
Sirve para verificar qué tan bien aprendió y si generaliza correctamente.

In [15]:
train_score = complete_pipeline.score(X_train, y_train)
test_score = complete_pipeline.score(X_test, y_test)

print(f"Exactitud entrenamiento: {train_score:.4f}")
print(f"Exactitud prueba: {test_score:.4f}")

Exactitud entrenamiento: 1.0000
Exactitud prueba: 1.0000


## Guardado del modelo.
Se importa `joblib` y se guardan el pipeline entrenado y el `LabelEncoder` en archivos `.pkl`.  
Esto permite reutilizar el modelo sin tener que reentrenarlo.  

In [16]:
import joblib

joblib.dump(complete_pipeline, 'pipeline_completo_personalizado.pkl')
joblib.dump(le_target, 'label_encoder.pkl')
print("Archivos guardados:")
print("- pipeline_completo_personalizado.pkl")
print("- label_encoder.pkl")

Archivos guardados:
- pipeline_completo_personalizado.pkl
- label_encoder.pkl
